<a href="https://colab.research.google.com/github/Nanangk/Bootcamp_Python_dari_SanberCode/blob/master/Week%204%20-%20Klasifikasi%20/Tugas_Harian_4_Minggu_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1> Quiz 1 Pemahaman tentang Model Selection</h1>

Jelaskan dengan bahasa kalian sendiri

1. Apa itu Bias dan Variance?
2. Apa itu Overfitting dan Underfitting?
3. Apa yang bisa kita lakukan untuk mengatur kompleksitas dari model?
4. Bagaimana model yang baik?
5. Kapan kira2 kita menggunakan GridSearchcv dan kapan menggunakan RandomizedSearchCV?

#Jawab
1. Variance error krn model terlalu kompleks, sdngkan bias model terlalu simple
2. Overfiting adalah salah 1 tipe yang menyebabkan model beforma buruk,memiliki training skor besar tapi tesnya kecil, sdngkan underfiting training skor dan testnya kecil
3. memilih hyper parameter untuk model kita supaya optimal
4. skor datatestnya besar dan skor data trainingnya besar
5. GridsearchCV semua kombinasi bisa lama dalam mentraing, 
RandomizedSearchCV bisa lebih cepat krn mengambil beberapa kombinasi secara random

<h1> Quiz 2 Praktis Model Selection</h1>

1. Bagi kedua data ini menjadi data training dan data test dengan test_size=0.25.
2. Gunakan algoritma KNN sebagai model classifier.
3. Gunakan fungsi GridSearchCV untuk hyperparameter tuning dan model selection.
4. jumlah fold bebas!, gunakan scoring 'roc_auc'
5. Definisikan kombinasi hyperparameter untuk model selection dengan GridSearchCV. kombinasi Hyperparameter bebas, baca lagi dokumentasi KNN di link berikut https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html untuk memahami lagi jenis2 hyperparameter di algorithma KNN.
6. Latih model terhadap data training.
7. Apa hyperparameter terbaik untuk kombinasi hyperparameter kalian?
8. Berapa score validasi terbaik dari model tersebut?
9. Prediksi probabilitasi output dari model yang telah di buat terhadap data test. note : gunakan method .predict_proba() untuk menghasilkan output probabilitas
10. Perhatikan bahwa hasil prediksi ada 2, dimana masing2 adalah nilai probabilitas untuk setiap class label. Ambil nilai probabilitas pasien phositive meninggal dalam waktu kurang dari 5 tahun. note : gunakan bantuan attirubte .classes_ untuk mengetahui urutan label dari hasil prediksi probabilitas.
11. Berapa nilai score roc_auc untuk data test?
12. Apakah model anda termasuk baik atau overtting atau underfitting?
13. Ulangi tahap di atas namun kali ini menggunakan algoritma DecisionTreeClassifier dan kalian bisa menggunakan RandomizedSearchCV apabila process training lama. pelajari algoritma DecisionTreeClassifier di linkberikut : https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html?highlight=decisiontreeclassifier#sklearn.tree.DecisionTreeClassifier
14. Bandingkan scorenya dengan Algoritma KNN, mana yang lebih baik?

Note : Data Science adalah experiment, sangat di dimungkinkan memerlukan beberapa kali percobaan untuk mendapatkan hasil yang terbaik! Happy Coding :)

In [0]:
import pandas as pd

url = 'https://raw.githubusercontent.com/jbrownlee/Datasets/master/haberman.csv'
list_cols = ['Age', "Patient's Years", "N_positive_ax", "survival_status"]
df = pd.read_csv(url, names=list_cols)

In [2]:
df.head()

,Age,Patient's Years,N_positive_ax,survival_status
0,30,64,1,1
1,30,62,3,1
2,30,65,0,1
3,31,59,2,1
4,31,65,4,1


In [0]:
x = df.drop('survival_status', axis=1)
y = df['survival_status']

In [0]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x,y, test_size = 0.25, random_state=21, stratify = y)

In [0]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
import numpy as np

In [0]:
model = KNeighborsClassifier()
param_grid = {
    'n_neighbors' : np.arange(5,30),
    'weights':['uniform', 'distance']
}
gscv = GridSearchCV(model, param_grid, cv=10, scoring='roc_auc')


In [12]:
gscv.fit(X_train, y_train)

GridSearchCV(cv=10, error_score=nan,
             estimator=KNeighborsClassifier(algorithm='auto', leaf_size=30,
                                            metric='minkowski',
                                            metric_params=None, n_jobs=None,
                                            n_neighbors=5, p=2,
                                            weights='uniform'),
             iid='deprecated', n_jobs=None,
             param_grid={'n_neighbors': array([ 5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21,
       22, 23, 24, 25, 26, 27, 28, 29]),
                         'weights': ['uniform', 'distance']},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='roc_auc', verbose=0)

In [13]:
gscv.best_params_

{'n_neighbors': 8, 'weights': 'distance'}

In [14]:
gscv.best_score_

0.7094012605042017

In [0]:
y_pred_proba = gscv.predict_proba(X_test)
y_pred_proba

In [18]:
gscv.classes_

array([1, 2])

In [0]:
y_pos_pred = y_pred_proba[:,1]

In [21]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test, y_pos_pred)

0.6166666666666667

In [23]:
y_pos_pred2 = y_pred_proba[:,0]
roc_auc_score(y_test, y_pos_pred2)

0.3833333333333333

In [0]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import RandomizedSearchCV

In [0]:
tree = DecisionTreeClassifier()
param_grid_dt = {
    'criterion': ['gini','entropy'],
    'max_depth' : np.arange(1,3),
    'min_samples_split':np.arange(2,100,20),
    'max_features': np.arange(1,3),
    'class_weight': [{1:x,2:1-x} for x in np.arange(0.1, 0.99, 0.1)]
}

In [39]:
gscv_dt = RandomizedSearchCV(tree, param_grid_dt, cv=10, scoring='roc_auc', n_iter=50)
gscv_dt.fit(X_train, y_train)

RandomizedSearchCV(cv=10, error_score=nan,
                   estimator=DecisionTreeClassifier(ccp_alpha=0.0,
                                                    class_weight=None,
                                                    criterion='gini',
                                                    max_depth=None,
                                                    max_features=None,
                                                    max_leaf_nodes=None,
                                                    min_impurity_decrease=0.0,
                                                    min_impurity_split=None,
                                                    min_samples_leaf=1,
                                                    min_samples_split=2,
                                                    min_weight_fraction_leaf=0.0,
                                                    presort='deprecated',
                                                    random_state=None,
         

In [40]:
gscv_dt.best_score_

0.6534751400560224

In [41]:
gscv_dt.best_params_

{'class_weight': {1: 0.1, 2: 0.9},
 'criterion': 'gini',
 'max_depth': 1,
 'max_features': 2,
 'min_samples_split': 62}

In [44]:
y_pred = gscv_dt.predict_proba(X_test)[:,1]
roc_auc_score(y_test,y_pred)

0.6232456140350877